# Run the final state algorithm on a ROOT HSDATA file

First create an object of your final state class. You may configure it if you want to limit the number of particles in an event or set to just run on generated simulation data.
This notebook is equivalent to the RunFSRootXXX.C macro which will be in the code directory.
Again just change XXX for your class name.
The string arguements set which particles you would like to use their pre-determined PID for and which you would like to be inclusive (i.e. any number of thse particles).
**Example**, 
1. for tagged photon experiment I might want fs("Beam","Beam")
This means use predetermined photon ID (i.e. I know it came from the tagger) and I will analyse events with any number of tagger photons.
2. fs("e-","proton:pi+:pi-")  i.e. use predetemined electron PID and just determine hadron species based on charge and try all combinations in events with any number of proton,pi+,pi-, but with the number of electrons defined for the topology
3. fs("NONE","ALL") => do not use any PID info other than charge, analyse events with any number of particles (this can be limited by SetMaxParticles)

In [ ]:
gROOT->ProcessLine(".x LoadPi2.C+"); //Load the classes

In [ ]:
 Pi2 fs("NONE","ALL"); //string arguments => PID, INCLUSIVE
  // fs.SetGenerated(); //just analyse generated branch
  // fs.SetMaxParticles(10); //max number of particles of any 1 type
 

Create an output tree. Here we will use a FiledTree object which is a wrapper class to a tree in a file. We will also create this as a smart pointer so we can delete it at the end by calling reset(), this will save the tree to disk. The Filed tree needs 2 strings ,1 a treename and the other the file name. You should give the full path for where you would like the files saved. It will overwrite any existing files. 

In [3]:
  //strings = treename and filename (give full path)
 fs.CreateFinalTree("FinalTree","/work/dump/c12test/OUTPUT.root");

Here we add the cuts for each particle. This is done via a HS::Cuts object which gets propogated to each particle in each topology. To specify any type of cut behaviour you need to generate your own Cuts Method. Here we just use a predefined delta time cut but you could make your won class which has momentum dependence as well.

Here I use a predefined CLAS12DeltaTime cuts class for PID. It takes 4 arguments which are cuts centred on zero for FT, FD, CD, FDCAL (for neutra) respectively.

I define specific cuts for proton and electron, while restrict other particles (here just pions) to only be in CD.

In [4]:
auto cutsman=make_shared<ParticleCutsManager>();
auto eCut=make_shared<CLAS12DeltaTime>(4,4,0,0); //4ns FT,FD,
auto protCut=make_shared<CLAS12DeltaTime>(0,5,5,0); //5ns FD,CD
auto otherCut=make_shared<CLAS12DeltaTime>(0,5,0,0); //5ns FD
cutsman->AddParticleCut("e-",eCut); //assign to manager
cutsman->AddParticleCut("proton",protCut); //assign to manager
cutsman->SetDefaultCut(otherCut); //assign to manager
cutsman->ConfigureCuts(&fs);   

Info ParticleCutsManager AddParticleCut set cut for 0
Info ParticleCutsManager AddParticleCut set cut for 11
Info ParticleCutsManager AddParticleCut set cut for 2212


Now register the cuts manager as a post topology function action. This will be applied after the Topo_0 etc functions when start times etc have been determined.

In [5]:
fs.RegisterPostTopoAction(cutsman);

Now we configure the input data. Here we use a ROOT file presaved in HSDATA format. It is possible to analyse other dataformats via the DataManager class.

**Particle Monitoring** here we are going to make seperate trees for each topology and only keep the detected particle information in each one. The string given to TreePrepManager should be the full path to the output file directory. 
Note as this is registered as a post work action all cuts, including any in Pi2::Kinematics, will be applied.

In [6]:
auto treeman=make_shared<ParticleDataManager>("/work/dump/c12test/particleTrees");
treeman->ConfigureTreeParticles(&fs); //propogate through topologies
treeman->AddFinal();
fs.RegisterPostWorkAction(treeman); //register post-work i.e. after

ParticleDataManager OutDir /work/dump/c12test/particleTrees cannot make this directory, you may need to delete it first or make sure its parent directory exists
NExt topo 
TreeParticleData:: Adding Electron
TreeParticleData:: Adding Proton
TreeParticleData:: Adding Pip
TreeParticleData:: Adding Pim
HS::TreeParticleData::SetBranches() Setting branches...
Register
done 
NExt topo 
TreeParticleData:: Adding Electron
TreeParticleData:: Adding Proton
TreeParticleData:: Adding Pip
HS::TreeParticleData::SetBranches() Setting branches...
Register
done 
NExt topo 
TreeParticleData:: Adding Electron
TreeParticleData:: Adding Proton
TreeParticleData:: Adding Pim
HS::TreeParticleData::SetBranches() Setting branches...
Register
done 
NExt topo 
TreeParticleData:: Adding Electron
TreeParticleData:: Adding Pip
TreeParticleData:: Adding Pim
HS::TreeParticleData::SetBranches() Setting branches...
Register
done 


Now I want to analyse a hipo file so I am going to use the datamanager HipoTrigger which is for full banks files. If I want to analyse just DSTs I should use HipoDST instead of HipoTrigger

In [7]:
 //create datamanager
  auto dm=std::make_shared<HipoToolsReader>();

  //And make a chain of data files
  TChain chain("HSParticles");
  chain.Add("/home/dglazier/fastdata/skim3_3933.hipo");
  dm->InitChain(&chain);

  //connect FinalState to Data by moving the pointer
  fs.SetDataManager(dm);


*****>>>>> compiled with c++11 support.
************************************************
*         >=<         ---------------------    *
*    ,.--'  ''-.      HIPO 3.0 I/O Library     *
*    (  )  ',_.'      Jefferson National Lab   *
*     Xx'xX           Date: 10/27/2017         *
************************************************

---> resizing internal compressed buffer size to from 0 to 4735980
 resizing internal buffer from 0 to 8379472
 read record =        28388 next      4759304  nevenets = 1845 length = 4730916

*****>>>>> compiled with c++11 support.
---> resizing internal compressed buffer size to from 0 to 33396
 resizing internal buffer from 0 to 27801
compression type = 0 data length = 28273
** error ** entry DETECTOR::Hits int schema hitPosition has undefined type.
** error ** entry DETECTOR::Hits int schema matchPosition has undefined type.
  covmatrix::init REC::CovMat
 calorimeter::init REC::Calorimeter
 scintilator::init REC::Scintillator
 cherenkov::init REC::Chere

Info in <DataManager::InitChain>:  Will proceess all 1 files in chain


Analyse all the events. Not you may prefer just looping over the data manager directly via while(dm->ReadEvent()) fs->Process();
You can also supply a number of events to analyse e.g. fs.ProcessData(100);

In [8]:
fs.ProcessData(); //No number given, analyse all events in chain

0 0
---> resizing internal compressed buffer size to from 4735980 to 4745980
 resizing internal buffer from 8387876 to 8380705
100000
 ProcessData event 100000
200000
 ProcessData event 200000
---> resizing internal compressed buffer size to from 4745980 to 4756676
300000
 ProcessData event 300000
400000
 ProcessData event 400000
500000
 ProcessData event 500000
600000
 ProcessData event 600000
700000
 ProcessData event 700000
800000
 ProcessData event 800000
900000
 ProcessData event 900000
1000000
 ProcessData event 1000000
1100000
 ProcessData event 1100000
1200000
 ProcessData event 1200000
1300000
 ProcessData event 1300000
1400000
 ProcessData event 1400000
1500000
 ProcessData event 1500000
1600000
 ProcessData event 1600000
1700000
 ProcessData event 1700000
1800000
 ProcessData event 1800000
1900000
 ProcessData event 1900000
2000000
 ProcessData event 2000000
2100000
 ProcessData event 2100000
2200000
 ProcessData event 2200000
2300000
 ProcessData event 2300000
2400000
 Proc

In [9]:
%jsroot

Draw some variables from your tree.

In [10]:
TCanvas c1;
fs.FinalTree()->Draw("MissMass2>>h1(100,-0.5,0.5)","Topo==2");
fs.FinalTree()->Draw("MissMass2","Topo==0","same");
fs.FinalTree()->Draw("MissMass2","Topo==1","same");
fs.FinalTree()->Draw("MissMass2","Topo==3","same");
//h1->SetMinimum(0);
c1.Draw();

In [11]:
TCanvas c3;
c3.Divide(2,2);
c3.cd(1);
treeman->GetParticleData(0)->GetTree()->Draw("ElectronP:ElectronTime>>h1(100,-10,10,100,0,12)","","col1");
c3.cd(2);
treeman->GetParticleData(0)->GetTree()->Draw("ProtonP:ProtonTime>>h2(100,-10,10,100,0,10)","","col1");
c3.cd(3);
treeman->GetParticleData(0)->GetTree()->Draw("PipP:PipTime>>h3(100,-10,10,100,0,10)","","col1");
c3.cd(4);
treeman->GetParticleData(0)->GetTree()->Draw("PimP:PimTime>>h4(100,-10,10,100,0,10)","","col1");
c3.Draw();

And remember to save the tree!

In [12]:
fs.EndAndWrite();

       tree name FinalTree 3276473 /work/dump/c12test/OUTPUT.root
       tree name ParticleVars 165927 /work/dump/c12test/particleTrees/ParticleVariables_0.root
       tree name ParticleVars 996479 /work/dump/c12test/particleTrees/ParticleVariables_1.root
       tree name ParticleVars 1493894 /work/dump/c12test/particleTrees/ParticleVariables_2.root
       tree name ParticleVars 620173 /work/dump/c12test/particleTrees/ParticleVariables_3.root
